<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/AnalisisDeDatos/2_Pivot_Table_y_Joins/clase-2-ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>


# Ejercicios Pandas II

In [62]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

## Parámetros y extracción de datos

In [63]:
ESTIMACIONES_PATH = "./data/estimaciones-agricolas-al-30-09-2019.csv"

In [64]:
converters = {
    "id_provincia": lambda x: str(x).zfill(2),
    "id_departamento": lambda x: str(x).zfill(3),
}

estimaciones = pd.read_csv(ESTIMACIONES_PATH, converters=converters, encoding='ISO-8859-1', low_memory=False)

estimaciones.head()

,id_provincia,provincia,id_departamento,departamento,id_cultivo,cultivo,id_campaña,campaña,sup_sembrada,sup_cosechada,produccion,rendimiento
0,06,Buenos Aires,854,25 de Mayo,1.0,Ajo,1,1969/1970,3,3,10,3333
1,06,Buenos Aires,014,Adolfo Gonzales Chaves,1.0,Ajo,1,1969/1970,15,15,82,5467
2,06,Buenos Aires,021,Alberti,1.0,Ajo,1,1969/1970,5,5,7,1400
3,06,Buenos Aires,028,Almirante Brown,1.0,Ajo,1,1969/1970,2,2,8,4000
4,06,Buenos Aires,077,Arrecifes,1.0,Ajo,1,1969/1970,6,6,15,2500


In [65]:
estimaciones.shape

(153889, 12)

In [66]:
estimaciones.columns

Index(['id_provincia', 'provincia', 'id_departamento', 'departamento',
       'id_cultivo', 'cultivo', 'id_campaña', 'campaña', 'sup_sembrada',
       'sup_cosechada', 'produccion ', 'rendimiento'],
      dtype='object')

In [67]:
new_columns = {col: col.strip() for col in estimaciones.columns}

# Renombrar las columnas del DataFrame
estimaciones = estimaciones.rename(columns=new_columns)

## Ejercicio 1 

Explorar descriptivamente el dataset de estimaciones agrícolas (https://datos.gob.ar/dataset/agroindustria-agricultura---estimaciones-agricolas) usando los elementos aprendidos en la clase 2, respondiendo las siguientes preguntas o mostrando:

* ¿Cuál es la producción total en toneladas estimada de todos los cultivos en la Argentina, cada año?
* ¿Cómo se distribuye el rendimiento (kg por hectárea) estimado promedio del cultivo de soja entre los departamentos que la siembran? ¿Qué tipo de distribución probabilística tiene? ¿Cuál es (y en qué departamento está) el rendimiento mínimo, mediano y máximo? ¿Cuál es el rendimiento promedio? ¿Hay valores potencialmente anómalos?
* ¿Cómo se distribuye la superficie sembrada de Papa? ¿Es una distribución probabilística similar a la de la soja? ¿El rendimiento en kg producidos por hectárea es similar entre los departamentos que la cultivan, se distribuye más o menos "equitativamente" que el de la soja? ¿Hay valores potencialmente anómalos?
* Generá una tabla donde cada cultivo sea una columna, cada fila un año (una campaña) y los valores sean la producción total.

In [68]:
estimaciones["cultivo"].unique()

array(['Ajo', 'Algodón', 'Alpiste', 'Arroz', 'Avena', 'Banana',
       'Caña de azúcar', 'Cártamo', 'Cebada cervecera',
       'Cebada forrajera', 'Cebolla total', 'Centeno', 'Girasol', 'Limón',
       'Lino', 'Maíz', 'Mandarina', 'Maní', 'Mijo', 'Naranja',
       'Papa total', 'Pomelo', 'Poroto seco', 'Soja total', 'Sorgo', 'Té',
       'Trigo total', 'Tung', 'Yerba mate', 'Trigo candeal', 'Colza',
       'Jojoba', 'Soja 1ra', 'Soja 2da', 'Cebada total', 'Poroto total',
       'Arveja', 'Garbanzo', 'Lenteja', 'Poroto alubia', 'Poroto negro',
       'Poroto otros'], dtype=object)

In [69]:
estimaciones["campaña"].unique()

array(['1969/1970', '1970/1971', '1971/1972', '1972/1973', '1973/1974',
       '1974/1975', '1975/1976', '1976/1977', '1977/1978', '1978/1979',
       '1979/1980', '1980/1981', '1981/1982', '1982/1983', '1983/1984',
       '1984/1985', '1985/1986', '1986/1987', '1987/1988', '1988/1989',
       '1989/1990', '1990/1991', '1991/1992', '1992/1993', '1993/1994',
       '1994/1995', '1995/1996', '1996/1997', '1997/1998', '1998/1999',
       '1999/2000', '2000/2001', '2001/2002', '2002/2003', '2003/2004',
       '2004/2005', '2005/2006', '2006/2007', '2007/2008', '2008/2009',
       '2009/2010', '2010/2011', '2011/2012', '2012/2013', '2013/2014',
       '2014/2015', '2015/2016', '2016/2017', '2017/2018', '2018/2019',
       '2019/2020', '2020/2021', '2021/2022', '2022/2023'], dtype=object)

## Producción total anual

In [70]:
estimaciones[["campania_inicio","campania_fin"]] = estimaciones.campaña.str.split("/", n=1,expand=True)

In [71]:
estimaciones["campania_inicio"] = estimaciones["campania_inicio"].astype(int)
estimaciones["campania_fin"] = estimaciones["campania_fin"].astype(int)

In [73]:
estimaciones["produccion"] = estimaciones["produccion"].fillna(0)
estimaciones["produccion"].replace('SD', 0, inplace=True)
# pasamos los valores a millones
estimaciones["produccion"] = estimaciones.produccion.astype(int)

/var/folders/ph/5sq6jlj161g1_2t838crnx6c0000gp/T/ipykernel_568/789318932.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  estimaciones["produccion"].replace('SD', 0, inplace=True)


In [74]:
# 1) ¿Cuál es la producción total en toneladas estimada de todos los cultivos en la Argentina, cada año?
prod_total_por_anio = estimaciones.pivot_table(
    index=["campania_fin"],
    values=["produccion"],
    aggfunc="sum"
)

prod_total_por_anio.sort_values("campania_fin", ascending=False)
prod_total_por_anio.tail()

,produccion
campania_fin,
2019,203090043
2020,190846551
2021,187573928
2022,187668131
2023,120831981
